![](./images/day10.png)

In [208]:
tiles = open('./data/input_day10.txt').read().splitlines()
grid = [list(l) for l in tiles]
grid[0][:5]

['|', '-', 'F', '7', '|']

In [209]:

def find_start(grid: list[list[str]]) -> tuple[int, int]:
    for i, row in enumerate(grid):
        for j, col in enumerate(row):
            if col == 'S':
                return (i, j)
            

def follow_path(start_coords: tuple[int, int], next_coords: tuple[int, int], grid: list[list[str]]) -> list[tuple[int, int]]:
    path = []
    while True:
        row, col = next_coords
        direction = tuple([n - s for n,s in zip(next_coords, start_coords)])
        pipe = grid[row][col]
        if pipe == "|" and direction in ((1, 0), (-1, 0)):
            path.append(next_coords)
        elif pipe == "-" and direction in ((0, 1), (0, -1)):
            path.append(next_coords)
        elif pipe == "L" and direction in ((1, 0), (0, -1)):
            path.append(next_coords)
            direction = (0, 1) if direction == (1, 0) else (-1, 0)
        elif pipe == "J" and direction in ((1, 0), (0, 1)):
            path.append(next_coords)
            direction = (0, -1) if direction == (1, 0) else (-1, 0)
        elif pipe == "7" and direction in ((0, 1), (-1, 0)):
            path.append(next_coords)
            direction = (1, 0) if direction == (0, 1) else (0, -1)
        elif pipe == "F" and direction in ((0, -1), (-1, 0)):
            path.append(next_coords)
            direction = (1, 0) if direction == (0, -1) else (0, 1)
        elif pipe == "S":
            path.append(next_coords)
            return path
        else:
            break
        
        start_coords = next_coords
        next_coords = (row + direction[0], col + direction[1])
        if any([n < 0 for n in next_coords]):
            break
    return []
            

def day10_part1(grid: list[list[str]]) -> int:
    start_coords = find_start(grid)
    paths = []
    for ra, ca in ((0, 1), (0, -1), (1, 0), (-1, 0)):
        next_coords = (start_coords[0] + ra, start_coords[1] + ca)
        print(len(paths))
        print("start:", next_coords)
        if any([n < 0 for n in next_coords]):
            continue
        path = follow_path(start_coords, next_coords, grid)
        if len(path) > 0:
            paths.append(path)
    middles = []
    for path in paths:
        size = len(path)
        middles.append(size // 2)
    return max(middles)


day10_part1(grid)

0
start: (86, 90)
1
start: (86, 88)
2
start: (87, 89)
2
start: (85, 89)


7012

## Part 2

In [205]:
tiles = open('./data/input_day10.txt').read().splitlines()
grid = [list(l) for l in tiles]
grid[0][:5]

['|', '-', 'F', '7', '|']

In [206]:
def is_point_inside_path(point: tuple[int, int], path: list[tuple[int, int]]) -> bool:
    x, y = point
    n = len(path)
    inside = False

    for i in range(n):
        x1, y1 = path[i]
        x2, y2 = path[(i + 1) % n]  # Wrap around to the start of the list

        if min(y1, y2) < y <= max(y1, y2) and x <= max(x1, x2):
            if y1 != y2:
                xinters = (y - y1) * (x2 - x1) / (y2 - y1) + x1
            if x1 == x2 or x <= xinters:
                inside = not inside

    return inside

In [203]:
def get_next_direction(pipe: str, direction: tuple[int, int]) -> tuple[int, int]:
    if pipe == "|" and direction in ((1, 0), (-1, 0)):
        return direction
    if pipe == "-" and direction in ((0, 1), (0, -1)):
        return direction
    if pipe == "L" and direction in ((1, 0), (0, -1)):
        direction = (0, 1) if direction == (1, 0) else (-1, 0)
        return direction
    if pipe == "J" and direction in ((1, 0), (0, 1)):
        direction = (0, -1) if direction == (1, 0) else (-1, 0)
        return direction
    if pipe == "7" and direction in ((0, 1), (-1, 0)):
        direction = (1, 0) if direction == (0, 1) else (0, -1)
        return direction
    if pipe == "F" and direction in ((0, -1), (-1, 0)):
        direction = (1, 0) if direction == (0, -1) else (0, 1)
        return direction
    return direction
    

def follow_path(start_coords: tuple[int, int], next_coords: tuple[int, int], grid: list[list[str]]) -> list[tuple[int, int]]:
    path = []
    while True:
        row, col = next_coords
        direction = tuple([n - s for n,s in zip(next_coords, start_coords)])
        pipe = grid[row][col]
        direction = get_next_direction(pipe, direction)
        if pipe == "S":
            path.append(next_coords)
            return path
        if pipe in ("|", "-", "L", "J", "7", "F"):
            path.append(next_coords)
        else:
            break
        
        start_coords = next_coords
        next_coords = (row + direction[0], col + direction[1])
        if any([n < 0 for n in next_coords]):
            break
    return []


def find_enclosed_tiles(path: list[tuple[int, int]], grid: list[list[str]]) -> int:
    candidates = []
    for row in range(0, len(grid)):
        for col in range(0, len(grid[0])):
            inside = is_point_inside_path((row, col), path)
            if (row, col) not in path and inside:
                candidates.append((row, col))
    
    return candidates
        
        
def day10_part2(grid: list[list[str]]) -> int:
    start_coords = find_start(grid)
    paths = []
    for ra, ca in ((0, 1), (0, -1), (1, 0), (-1, 0)):
        next_coords = (start_coords[0] + ra, start_coords[1] + ca)
        print(len(paths))
        print("start:", next_coords)
        if any([n < 0 for n in next_coords]):
            continue
        path = follow_path(start_coords, next_coords, grid)
        if len(path) > 0:
            paths.append(path)
    tiles_enclosed = 0
    unique_paths = []
    for path in paths:
        path.pop()
        if path not in unique_paths and path[::-1] not in unique_paths:
            unique_paths.append(path)

    paths = [[start_coords] + path for path in unique_paths]
    path = max(paths, key=lambda x: len(x))
    
    tiles_enclosed += len(find_enclosed_tiles(path, grid))
    return tiles_enclosed

day10_part2(grid)

0
start: (86, 90)
1
start: (86, 88)
2
start: (87, 89)
3
start: (85, 89)


395